# CDS services in Jupyter notebooks

Katarina A. Lutz¹, Manon Marchand¹

1. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France

***



## Introduction

Welcome to a Jupyter notebook demonstrating how to access some of the services provided by the Strasbourg astronomical Data Center ([CDS](https://cdsweb.u-strasbg.fr/)) with Python.  

In this tutorial, we will:
1. gather information on a specific object using [![Simbad](https://custom-icon-badges.demolab.com/badge/Simbad-gray.svg?logo=simbad&logoColor=lightblue&logoWidth=20)](https://simbad.u-strasbg.fr/simbad/ "https://simbad.u-strasbg.fr/simbad/"), 
2. visualise it through survey images using the [![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml") Lite widget, 
3. find and download a catalogue from [![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR"),
4. overlay the sources of this catalog on the sky view in the Aladin Lite widget, 
5. cross-match the sources with a large catalogue using [![Xmatch](https://custom-icon-badges.demolab.com/badge/Xmatch-gray.svg?logo=xmatch&logoColor=blue&logoWidth=20)](http://cdsxmatch.u-strasbg.fr/ "http://cdsxmatch.u-strasbg.fr/"). 

In [1]:
# Astronomy tools
import astropy.units as u

# Access astronomical databases
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch

# Sky visualization
from ipyaladin import Aladin

## Step #1: Gather information on NGC4038 


[![Simbad](https://custom-icon-badges.demolab.com/badge/Simbad-gray.svg?logo=simbad&logoColor=lightblue&logoWidth=20)](https://simbad.u-strasbg.fr/simbad/ "https://simbad.u-strasbg.fr/simbad/")


As a first step, we check what information is directly available from the <ins>Simbad</ins> python module. Let's use the method `query_object` to gather information. Note that you can look at the methods available for the `Simbad` class with the `help(Simbad)` command:

In [2]:
Simbad.query_object("NGC4038")

main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,matched_id
,deg,deg,mas,mas,deg,,,
object,float64,float64,float32,float32,int16,str1,object,object
NGC 4038,180.47084166666667,-18.86758888888889,--,--,--,I,2006AJ....131.1163S,NGC 4038


We get basic information, like the position of `NCG4038`. This search function can be personalized by adding fields to the response. Let's list the possible fields:

In [3]:
fields = Simbad.list_votable_fields()
fields

name,description,type
object,object,object
mesDiameter,Collection of stellar diameters.,table
mesPM,Collection of proper motions.,table
mesISO,Infrared Space Observatory (ISO) observing log.,table
mesSpT,Collection of spectral types.,table
allfluxes,"all flux/magnitudes U,B,V,I,J,H,K,u_,g_,r_,i_,z_",table
ident,Identifiers of an astronomical object,table
flux,Magnitude/Flux information about an astronomical object,table
mesPLX,Collection of trigonometric parallaxes.,table
otypedef,all names and definitions for the object types,table


We get a list of the available fields we could add to the SIMBAD response.

In [4]:
simbad = Simbad()  # we create an instance that we can personalize
simbad.add_votable_fields("otype", "alltypes", "dim")
info_simbad = simbad.query_object("NGC4038")
info_simbad

main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,otype,otypes,galdim_majaxis_prec,galdim_majaxis,galdim_minaxis,galdim_qual,galdim_bibcode,galdim_wavelength,galdim_angle,galdim_minaxis_prec,matched_id
,deg,deg,mas,mas,deg,,,,,,arcmin,arcmin,,,,deg,,
object,float64,float64,float32,float32,int16,str1,object,object,object,int16,float32,float32,str1,object,str1,int16,int16,object
NGC 4038,180.47084166666667,-18.86758888888889,--,--,--,I,2006AJ....131.1163S,EmG,AG?|EmG|G|GiP|HI|IG|PaG|Rad,1,5.2,3.1,D,2007ApJS..173..185G,,80,1,NGC 4038


We added the object type with the field `otype`. We now know the main object type of NGC4038: "Emission line Galaxy" `EmG`.

We can also read the object dimensions, that we added with the field `dim`. (major axis `galdim_majaxis`, minor axis `galdim_minaxis`, inclination angle `galdim_angle`..., and the related bibliographic reference `galdim_bibcode` where the dimension information was extracted).

We can also see the full list of object types attributed to NGC4038 in the past (ex: `HI` HI (21cm) Source, `G` Galaxy, `GiP` Galaxy in Pair, `Rad` Radio Source...). This comes from the fact that an object can have a lot of descriptions, a galaxy can very well also be a radio source. Sometimes, our understanding evolved with time and some of these former types won't be compatible. The descriptions and relations between object types can be explored on [Simbad's webpages](https://simbad.cds.unistra.fr/guide/otypes.htx).

As we saw that NGC4038 is a galaxy in a pair, let's search for the second galaxy in the pair. Let's first identify the pair by looking into `NGC4038`'s parents:

In [5]:
simbad.reset_votable_fields()
simbad.add_votable_fields("otype")
siblings = simbad.query_hierarchy("NGC4038", hierarchy="parents")
siblings

main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,otype
,deg,deg,mas,mas,deg,,,
object,float64,float64,float32,float32,int16,str1,object,object
NAME Antennae,180.47154166666667,-18.8772,--,--,--,I,2006AJ....131.1163S,IG
NAME NGC 4038 Group,179.98833333333332,-19.2725,--,--,--,O,1993A&AS..100...47G,GrG
[T2015] nest 100219,179.8429,-18.9931,--,--,--,O,2015AJ....149..171T,GrG
[TSK2008] 409,179.85802908705062,-19.40075160642234,--,--,--,O,2013AJ....146...86T,GrG


The Antennae Interacting Galaxies (of object type `IG`) ais the parent. Let's now look at its children. We also add a filter on the type of objects with the `criteria` argument in which we select any type of galaxies (that's the notation `G..`)

In [6]:
siblings = simbad.query_hierarchy(
    "Antennae", hierarchy="children", criteria="otype='G..'"
)
siblings

main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,otype
,deg,deg,mas,mas,deg,,,
object,float64,float64,float32,float32,int16,str1,object,object
NGC 4038,180.47084166666667,-18.86758888888889,--,--,--,I,2006AJ....131.1163S,EmG
NGC 4039,180.47295999999997,-18.88619,--,--,--,O,2020MNRAS.494.1784A,GiP


You can learn more on the features of the Simbad python module here: https://astroquery.readthedocs.io/en/latest/simbad/simbad.html

and on the Simbad query criteria: http://simbad.cds.unistra.fr/simbad/sim-fsam

## Step #2: Visualise the source on a sky map

[![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml")

We now open the [Aladin Lite widget](https://github.com/cds-astro/ipyaladin "https://github.com/cds-astro/ipyaladin"). We set it first to center on the Antennae galaxies, have a **field of view** of 0.5 $^\circ$ and show color images from the Digitized Sky Survey ([DSS](https://archive.stsci.edu/cgi-bin/dss_form)). Typing `aladin` in the second line of code tells the notebook to display the widget. 

In [7]:
aladin = Aladin(target="Antennae", fov=0.5, survey="P/DSS2/color", height=600)
aladin

Aladin(survey='P/DSS2/color')

As with any Aladin Lite implementation, you can interact with this widget. Try to:
 - scroll with your mouse pointer on the image to zoom in and out,  
 - select other image surveys and manage the current view with <img src="Data/images/ipyaladin_layer.png" width="30">
 - look at another target using the search field <img src="Data/images/ipyaladin_search.png" width="30">
 - find more information on something with the target tool <img src="Data/images/target.png" width="30">
 - do a right click to discover more tools

You can also directly change the properties of the variable `aladin`. For example, to focus on M101 instead of the Antennae galaxies:

In [8]:
aladin.target = "M101"

Feel free to explore other targets and methods available for the Aladin widget.

## Step #3: Find and visualize additional data

[![Aladin](https://custom-icon-badges.demolab.com/badge/Aladin-gray.svg?logo=aladin&logoColor=purple&logoWidth=20)](https://aladin.cds.unistra.fr/aladin.gml "https://aladin.cds.unistra.fr/aladin.gml")
[![Simbad](https://custom-icon-badges.demolab.com/badge/Simbad-gray.svg?logo=simbad&logoColor=lightblue&logoWidth=20)](https://simbad.u-strasbg.fr/simbad/ "https://simbad.u-strasbg.fr/simbad/")
[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")

We want to look for data related to HII regions in these interacting galaxies. We make a query in <ins>Simbad</ins> for HII regions within 14 arcmins around the Antennae galaxies. 

In [9]:
simbad.reset_votable_fields()
# biblio contains the bibcodes of all papers where the SIMBAD team found the object
simbad.add_votable_fields("otype", "biblio")
table_simbad = simbad.query_region("Antennae", radius="14m", criteria="otype='HII'")
table_simbad

main_id,ra,dec,coo_err_maj,coo_err_min,coo_err_angle,coo_wavelength,coo_bibcode,otype,biblio
,deg,deg,mas,mas,deg,,,,
object,float64,float64,float32,float32,int16,str1,object,object,object
[ZBF2015] Arp244 37,180.46375,-18.861833333333333,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z
[ZBF2015] Arp244 219,180.47833333333335,-18.878833333333336,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z
[ZBF2015] Arp244 80,180.47083333333333,-18.867638888888887,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z
[ZBF2015] Arp244 38,180.45875,-18.86958333333333,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z
[ZBF2015] Arp244 196,180.47625000000002,-18.86675,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z
[ZBF2015] Arp244 126,180.47750000000002,-18.86788888888889,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z
[ZBF2015] Arp244 149,180.46958333333336,-18.87183333333333,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z
[ZBF2015] Arp244 130,180.48250000000002,-18.885194444444444,--,--,--,O,2014MNRAS.445.1412Z,HII,2014MNRAS.445.1412Z|2015MNRAS.451.1307Z


Once we have identified a bibliographic reference of interest, we can search the corresponding catalogs on <ins>VizieR</ins>. Here, the Biblio column has quite some occurrences of the reference `'2015MNRAS.451.1307Z'`.

Note that by default VizieR will only return 50 entries (as in the web interface). To get all sources, set `ROW_LIMIT=-1`.

A good practice is to check first what data are available by querying only a subsample, then customize your Vizier query with the useful rows and columns before downloading the entire catalog.

In [10]:
cat_viz = Vizier(row_limit=10).get_catalogs("2015MNRAS.451.1307Z")
cat_viz

TableList with 2 tables:
	'0:J/MNRAS/451/1307/HIIreg' with 15 column(s) and 10 row(s) 
	'1:J/MNRAS/451/1307/list' with 13 column(s) and 10 row(s) 

In the example above, we see that 2 tables are associated with the reference `2015MNRAS.451.1307Z`: a table called 'HIIreg' accessible by the index [0] and a table called 'list' accessible by the index [1]. The number of rows has been fixed by the parameter `row_limit=10`.

Let's check the 'HIIreg' table:

In [11]:
# print the metadata
cat_viz[0].meta

{'ID': 'J_MNRAS_451_1307_HIIreg',
 'name': 'J/MNRAS/451/1307/HIIreg',
 'description': 'Physical properties of the brightest HII regions in interacting galaxies (table A1) and isolated galaxies (table A2)'}

In [12]:
# print the table itself
cat_viz[0]

Seq,n_Name,Name,Nf,RAJ2000,DEJ2000,logLHa,R,logMHII,rhoHII,sigma,alpVir,EWHa,logAge,Simbad
,,,,,,log(1e-07W),pc,log(solMass),solMass / pc3,km / s,,0.1 nm,log(yr),
int16,str3,str7,int16,str10,str11,float32,int16,float32,float32,float32,float32,float32,float32,str6
1,int,NGC2146,3,06 18 39.4,+78 21 18.2,39.040,114,5.90,0.1270,15.0,40.0,159.79,6.770,Simbad
2,int,NGC2146,3,06 18 38.1,+78 21 26.8,38.830,116,5.80,0.0970,17.0,60.0,54.70,6.810,Simbad
3,int,NGC2146,3,06 18 44.6,+78 21 50.6,38.590,142,5.80,0.0550,11.0,30.0,125.30,6.780,Simbad
4,int,NGC2146,3,06 18 27.7,+78 22 21.0,38.470,155,5.81,0.0410,8.6,20.0,119.64,6.780,Simbad
5,int,NGC2146,3,06 18 35.0,+78 21 28.0,38.440,124,5.65,0.0560,15.0,80.0,103.35,6.790,Simbad
6,int,NGC2146,3,06 18 42.8,+78 21 58.1,38.410,136,5.70,0.0470,4.0,--,155.86,6.770,Simbad
7,int,NGC2146,3,06 18 39.8,+78 21 22.5,38.350,81,5.30,0.0960,15.0,100.0,109.47,6.790,Simbad
8,int,NGC2146,3,06 18 48.0,+78 21 43.3,38.320,144,5.69,0.0390,4.0,--,81.89,6.800,Simbad


As for every function in python, we can call `help(Vizier)` in a code cell and run it to print the documentation. 

We won't do it here as it is quite long, but if we did, we could have read that `columns` should be a list, `column_filters` a dictionary, and `row_limits` is an int for which -1 means infinity. Now we can customize our VizieR query by selecting columns, filtering to keep only the interacting galaxies (`n_Name='int'`), and keeping only the first region for each galaxy (`Seq=1`, for visualization purposes).

In [13]:
cat_hii = Vizier(
    columns=["Name", "RAJ2000", "DEJ2000", "logLHa", "R", "sigma"],
    column_filters={"Seq": "==1", "n_Name": "int"},
    row_limit=-1,
).get_catalogs("2015MNRAS.451.1307Z")[0]
cat_hii

Name,RAJ2000,DEJ2000,logLHa,R,sigma
,,,log(1e-07W),pc,km / s
str7,str10,str11,float32,int16,float32
NGC2146,06 18 39.4,+78 21 18.2,39.040,114,15.0
Arp244,12 01 54.6,-18 53 03.1,40.200,280,32.8
NGC520,01 24 33.3,+03 48 04.6,39.120,255,14.1
UGC3995,07 44 09.2,+29 14 51.3,40.290,270,43.0
NGC3788,11 39 41.4,+31 54 23.3,39.700,280,11.9
NGC3786,11 39 45.1,+31 55 46.3,39.860,300,16.5
NGC2782,09 14 05.3,+40 06 50.4,39.800,166,28.0
NGC2993,09 45 48.3,-14 22 05.9,40.070,258,12.0


We can also display the table on <ins>Aladin</ins>. 
We use `aladin.add_table`

In [14]:
aladin.add_table(cat_hii)

Go up to the <ins>Aladin Lite</ins> widget, zoom out to see all the sources, then rotate and zoom in to see the interacting galaxies. You can click on one source to display the information in the table. 

Change the view to the AllWISE Infrared survey, either in the widget:

`Base image layer -> The Wide-field Infrared Survey Explorer` 

or in the command line below.

You can find the list of available sky maps at https://aladin.u-strasbg.fr/hips/list

In [15]:
aladin.survey = "P/allWISE/color"

If you want to overlay the survey instead of replacing it:

`aladin.overlay_survey= 'P/allWISE/color'`

`aladin.overlay_survey_opacity = 0.5`

**Exercise:**

Retrieve the second table from Zaragoza-Cardiel+, 2015 (2015MNRAS.451.1307Z) containing data related to the image fits files and try to answer the questions:
- What are the typical sizes of the fits images?
- What is the approximate resolution of the images? [arcsec/pix]

In [16]:
# write your code here

## Step #5: Cross-match tables

[![Vizier](https://custom-icon-badges.demolab.com/badge/Vizier-gray.svg?logo=vizier&logoColor=orange&logoWidth=20)](https://vizier.cds.unistra.fr/viz-bin/VizieR "https://vizier.cds.unistra.fr/viz-bin/VizieR")
[![Xmatch](https://custom-icon-badges.demolab.com/badge/Xmatch-gray.svg?logo=xmatch&logoColor=blue&logoWidth=20)](http://cdsxmatch.u-strasbg.fr/ "http://cdsxmatch.u-strasbg.fr/")

We have HII data for a sample of interacting galaxies. Since we are also interested in the infrared fluxes we search for the Wide-field Infrared Survey Explorer ([AllWISE](https://wise2.ipac.caltech.edu/docs/release/allwise/)) data release. 

We want to match the sources in J/MNRAS/451/1307/HIIreg with the AllWISE source catalog that contains 747 million objects!

We use the <ins>X-Match</ins> python module for cross-identification of objects. It is particularly efficient and fast with very large catalogs, like AllWISE. All we need are the names of the catalogs, the names of the columns containing the coordinates of the sources, and the desired maximum distance for the match (optionally the area as well, otherwise all-sky is the default).

In [17]:
# What is the AllWISE catalogue reference in VizieR?
cat_allwise = Vizier.get_catalogs("AllWISE")
cat_allwise

TableList with 1 tables:
	'0:II/328/allwise' with 28 column(s) and 50 row(s) 

In [18]:
xmatch_zbf_wise = XMatch.query(
    cat1="vizier:J/MNRAS/451/1307/HIIreg",
    cat2="vizier:II/328/allwise",
    max_distance=10 * u.arcsec,
    colRA1="RAJ2000",
    colDec1="DEJ2000",
    colRA2="RAJ2000",
    colDec2="DEJ2000",
)
xmatch_zbf_wise

angDist,_RAJ2000,_DEJ2000,recno,Seq,n_Name,Name,Nf,RAJ2000,DEJ2000,logLHa,R,logMHII,rhoHII,sigma,alpVir,EWHa,logAge,Simbad,AllWISE,RAJ20002,DEJ20002,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M
arcsec,deg,deg,,,,,,,,log(1e-07W),pc,log(solMass),solMass.pc**-3,km / s,,0.1 nm,log(yr),,,deg,deg,arcsec,arcsec,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,,,,,,,,
float64,float64,float64,int32,int16,str3,str7,int16,str10,str11,float32,int16,float32,float32,float32,float32,float32,float32,object,str19,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,str4,str1,str4,str4,int32,int32,int32,int32,float32
9.439952,250.775,36.839056,1820,27,iso,NGC6207,0,16 43 06.0,+36 50 20.6,37.69,66,4.9,0.06,4.0,--,134.26,6.78,Simbad,J164305.83+365029.8,250.7743217,36.8416214,0.0374,0.0369,67.0,12.87,12.918,8.29,5.588,--,--,--,0.073,0.025,0.022,0.037,--,--,--,2499136301351044678,0000,3,3000,AAAA,10,40,359,42,--
6.754932,250.767917,36.834944,1821,28,iso,NGC6207,0,16 43 04.3,+36 50 05.8,37.66,101,5.1,0.031,4.0,--,22.27,6.91,Simbad,J164303.76+365003.8,250.7656686,36.8344125,0.0212,0.0201,135.0,10.792,11.24,7.028,4.232,12.866,12.603,12.523,0.03,0.019,0.013,0.021,0.043,0.056,0.058,2499136301351044679,hd00,3,n9n3,AAAA,-929,25,-1205,26,0.458
8.575258,250.767917,36.834944,1821,28,iso,NGC6207,0,16 43 04.3,+36 50 05.8,37.66,101,5.1,0.031,4.0,--,22.27,6.91,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--
5.99581,250.76375,36.833083,1822,29,iso,NGC6207,0,16 43 03.3,+36 49 59.1,37.66,87,5.0,0.039,4.0,--,20.6,6.93,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--
7.312318,250.76375,36.833083,1822,29,iso,NGC6207,0,16 43 03.3,+36 49 59.1,37.66,87,5.0,0.039,4.0,--,20.6,6.93,Simbad,J164303.76+365003.8,250.7656686,36.8344125,0.0212,0.0201,135.0,10.792,11.24,7.028,4.232,12.866,12.603,12.523,0.03,0.019,0.013,0.021,0.043,0.056,0.058,2499136301351044679,hd00,3,n9n3,AAAA,-929,25,-1205,26,0.458
3.802161,250.766667,36.832444,1823,30,iso,NGC6207,0,16 43 04.0,+36 49 56.8,37.66,91,5.1,0.036,4.0,--,8.9,7.09,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--
7.648264,250.766667,36.832444,1823,30,iso,NGC6207,0,16 43 04.0,+36 49 56.8,37.66,91,5.1,0.036,4.0,--,8.9,7.09,Simbad,J164303.76+365003.8,250.7656686,36.8344125,0.0212,0.0201,135.0,10.792,11.24,7.028,4.232,12.866,12.603,12.523,0.03,0.019,0.013,0.021,0.043,0.056,0.058,2499136301351044679,hd00,3,n9n3,AAAA,-929,25,-1205,26,0.458
9.762527,250.769167,36.832833,1825,34,iso,NGC6207,0,16 43 04.6,+36 49 58.2,37.49,57,4.7,0.06,4.0,--,18.26,6.98,Simbad,J164303.79+364959.7,250.7658199,36.8332538,0.0379,0.0338,158.6,10.953,10.936,7.758,5.483,--,--,--,0.023,0.022,0.024,0.066,--,--,--,2499136301351044647,d0d0,5,3200,AAAA,113,31,484,32,--


<h4 align="center">This is the end of the tutorial.</h4> 

<h4 align="center">Try to apply what you have learned here to your data sets and science cases! </h4>